## What is a “Dynamic Page”?

A **dynamic page** is a website where some or all of the content loads **after** the page appears, often using JavaScript.

**Examples:**
- Social media feeds
- Infinite scroll
- Content that appears only after you click or scroll

---

### Problem:

- `requests` and BeautifulSoup can only see the **initial HTML**—they miss content loaded later with JavaScript.

---

### Solution:

- We use a tool that controls a real web browser, just like a human.
- **Selenium** is the most popular tool for this job.

## Handle JavaScript-rendered Content with Selenium

### What is Selenium?

**Selenium** is a Python library that can:

- Open a real browser window (like Chrome or Firefox)
- Click buttons, scroll pages, type into boxes—anything a human can do!
- Wait for content to load, then grab the new HTML

### Why use Selenium?

It lets you see and scrape the page after all the JavaScript has finished loading content.

### Setting up ChromeDriver

1. Download ChromeDriver that matches your Chrome browser version
2. Place the chromedriver file in your project folder
    - **Alternatively:** Use the full file path in your code
3. Make sure ChromeDriver is accessible in your system PATH

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import csv